In [0]:
import os, time
from google.colab import files
from random import seed, shuffle
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions.multinomial import Multinomial
import torch

In [0]:
import pdb

Make sure to set hardware accelerator to GPU in settings

In [4]:
uploaded = files.upload()

Saving t8.shakespeare.txt to t8.shakespeare.txt


In [5]:
!rm -rf scribble-bot && git clone https://github.com/Chatha-Sphere/scribble-bot.git

Cloning into 'scribble-bot'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 75 (delta 39), reused 62 (delta 26), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [0]:
os.chdir('scribble-bot')

In [0]:

from corpora import CharEncoder, extract_shakespeare_data
from helpers import sample, make_sequences
from train import train
from model import CharRNN

In [0]:
os.chdir('..')

In [0]:
text, corpus = extract_shakespeare_data('t8.shakespeare.txt')
char_encoder = CharEncoder(corpus)
sequences = make_sequences(text)
sequences = char_encoder.label_sequences(sequences)

In [0]:
seed(1616)
shuffle(sequences)
n_training_sequences = int(.9 * len(sequences))
training = sequences[:n_training_sequences]
validation = sequences[n_training_sequences:]

In [0]:
hidden_size = 256
rnn = CharRNN(char_encoder.n_chars, hidden_size, n_rnn_layers=2, dropout = 0.5)

In [13]:
train(rnn, training, validation, epochs = 10, lr = 0.01, evaluate_per = 2, batch_size = 50)

GPU is available
epoch: 1/10 | time: 50s
training loss: 1.57
epoch: 2/10 | time: 50s
training loss: 1.45
validation loss: 1.39
epoch: 3/10 | time: 50s
training loss: 1.43
epoch: 4/10 | time: 51s
training loss: 1.35
validation loss: 1.33
epoch: 5/10 | time: 52s
training loss: 1.41
epoch: 6/10 | time: 51s
training loss: 1.33
validation loss: 1.31
epoch: 7/10 | time: 52s
training loss: 1.36
epoch: 8/10 | time: 52s
training loss: 1.33
validation loss: 1.30
epoch: 9/10 | time: 52s
training loss: 1.35
epoch: 10/10 | time: 51s
training loss: 1.34
validation loss: 1.29


Generating text off a short primer string

In [0]:
def sample2(model, encoder, size, prime_str, temperature, topk = None):
    """
    Randomly generate text from a trained model.

    Args:
        model (nn.module object): trained LSTM neural network
        encoder (CharEncoder object): contains character-to-integer mappings
        size (int): length of generated sample
        prime_str (str): input to initialize the network's hidden state
        temperature (float in range (0,1]): dampens the model's character probability distribution:
            values less than 1 make the model more conservative, giving additional weight to
            high-probability guesses and penalizing low-probability guesses. As temperature goes to
            zero, softmax becomes "argmax," and the most probable character is picked almost certainly.

    Returns:
        output_str (str): Randomly-generated text sample of length (size)
    """
    #deactivate training mode
    model.eval()
    #initialize output string
    output_str = prime_str
    #vectorize input string as a sequence of ints
    input_sequence = [encoder.char2int[char] for char in prime_str]
    #initialize hidden state to None
    hx = None
    for i in range(size): #generate characters of output string one at a time
        #get model output and hidden state (short-term and long-term memory)
        out, hx = model([input_sequence], hx)
        hx = tuple(h.detach() for h in hx)
        #ignore batch dimension because our batch is of size 1
        out = out.squeeze()
        #calculate log softmax over the possible characters in the corpus.
        #resulting log probabilities: more numerical stability for low probabilities,
        #
        #dist = (F.log_softmax(out/ temperature, dim=-1).data).exp()
        probs = F.softmax(out/ temperature, dim=-1).data
          
        #If probs are generated on a string of multiple characters,
        #keep prediction of next character only
        if len(probs.shape) > 1:
          probs = probs[-1,:]
        
        if topk is not None:
          values, indices = probs.topk(topk)
          if torch.cuda.is_available():
            zeros = torch.zeros(encoder.n_chars).cuda()
          else:
            zeros = torch.zeros(encoder.n_chars)
          probs =  torch.scatter(zeros, 0, indices, values)  
          
        next_char_ix = torch.multinomial(probs,1).item()

        input_sequence = [next_char_ix]

        output_str += encoder.int2char[next_char_ix]

    return output_str


In [87]:
print(sample2(rnn, char_encoder, 1000, prime_str='Ham.', temperature=.85, topk=9))

Ham.              Exeunt
ACT IV.
Mayol.
    Were you dear'd to cease alone,
    So shone our bold of makes a true stars of any hands to be master'd; and take them.
     Thou canst thee that is the house of thy soldiers.
    And, shall I speak with that. Good master, being thy dear,
    As soon thought to see.  
    To the grievous cavamies of here had more and a sport.
    Well, had mother?
    The mind in this will truncied with the heav'nly that we do place your common believing
    With the briberching of that special
    The best commend; but in such and defant
    Which will be death, to be to him.
  Cor. I shall not tell me we than when you speak the prisoner.
  Cap. What shall I thou hast tell me there was horn
    This sins, sweet sight; and you swear in
    a better be his sents and thee, of you,
    And that the patric of some spition, my leave and heads
    That we are talking mistress.
  MACBETH. This is my head from the bounteous side wingings my
    pow'r to say 'To the m

In [83]:
for topk in [10, 9, 8]:
  print(f"topk characters: {topk}")
  print(sample2(rnn, char_encoder, 1000, prime_str = "Ham.", temperature=.85,
               topk = topk))
  print('\n********\n')

topk characters: 10
Ham.
  CASSIUS. Ah, man.
    What sewashion is as there is better many a manner.
    Thou art well so stalk of their father.
  MUTILLUS. Ay, sir?
    What says he is?
    I do read.
  SECOND GENTLEMAN
    She calls me have to me not; I mean your masters or disposition must
    The foreed, and a game.
  MRS. FORD. We have murther, or alt, but the scape,
    The house of thy book would have a good day?
    The scorn here.                                                          Exit.
Scene V.
All the star.                                                                   [This banished prisoners
  PISANIO. It was this dear as sure
    In your beauty to be sometime,
    And make me for this dead conception is mad with his
    back one answer and so fortune.
                                                                                                               Re-enter MESSENGER and BARDOLPH]  In this contrivere is her
    strength billows in his hand to the hear

In [39]:
print("Generating off of primer string 'MACBETH.':")
for temp in [0.8, 0.85, .9]:
  print(f"temperature: {temp}")
  print(sample2(rnn, char_encoder, 1000, prime_str='MACBETH.', temperature=temp))
  
  print("\n********\n")

Generating off of primer string 'MACBETH.':
temperature: 0.8
MACBETH. Now, my liege, and did with the eyes;
    And, and I have please. I say 'Amen, you were this wall!
  PAROLLES. We'll follow me, if you were so for admirable
    Was speak.
  Captain, I have most son Arthur then.
    [To TROILUS in my lady to return.
  BRUTUS. I do of o'er;  
    And yet the common bed when's the sceptres
     Which they that did struight to you may have
    For dire I do to mother. Thou know'st.
  QUEEN MARGARET. Now she was his book; but Henry
    The tricks of the considerly praise.                                                                            Exit
                                                                          Exit
    The KING SENATOR'S heavens. 'Tis presently, let me not fortune
    With myself be common stones and noble threats
    Where you pass in my birth at their earth,
    To your harm;
    For such of them to an exile.
                                               

In [18]:
print("Generating off of primer string 'MACBETH.':")
for temp in [0.8, 0.85, .9]:
  print(f"temperature: {temp}")
  print(sample(rnn, char_encoder, 1000, prime_str='MACBETH.', temperature=temp))
  
  print("\n********\n")

Generating off of primer string 'MACBETH.':
temperature: 0.8
MACBETH. Instince he sleep- whether, or bear him to York. Will you follow
    That I to mercy were I the soldier to enery.
  WILLIAM. The travell of senses and other, here.
  Hot. Stay to the way; but, nor he is his bows.
    What was the youth be the duty of a honest fall
    That I mean to our bride and man. Here comes the pleasure, sir;
    By the Lord Savera, curse, and I will be all thy sweet word.
    Let me see them between my sorrows of Cassio.
  FIRST CITIZEN. A-money, as sure would see it.
                                                   Exeunt Gloucester.
  POSTHUMUS. Whither Edward  
    Will pleas'd him so the thing since I know with piece
    Of the body near meet you to bohe things in the
    business with thy strain, reasoning to this time
    To put your bloody friar and all the Greeter,
    from fines of eyes.
    I'll fain to please your virtuous time do dewicked.
  GRUMIO. And what is here redeem, marry,

Generating text off a long primer string

In [21]:
print("Generating off a longer primer string:")
#long_prime = "Pol. [aside] Though this be madness, yet there is a method in't.-\nWill You walk out of the air, my lord?"
long_prime = "Pol. [aside] Though this be madness, yet there is a method in't."
for temp in [0.83, 0.85, .88]:
  print(f"temperature: {temp}")
  print(sample(rnn, char_encoder, 1000, prime_str=long_prime, temperature=temp))
  
  print("\n********\n")

Generating off a longer primer string:
temperature: 0.83
Pol. [aside] Though this be madness, yet there is a method in't.
  KATHERINA. If you line to thy power, he comes stand a- did we have
    I'll forgive your pains understand in shadows
    Was satisfied not of battle that would to his desire
    And will be liver to the Majesty; and the King's
    honour; kunder with a beauty too fear'd
    Something but I think to this good worthiest king at good jewers bewiths, which they are to you the holy streets
    To make in, and Tholasture!
  PRINCESS OF FRADCEMANT. Where were you so? This is he, laughous'd forth 'Castle.
    What begins indeed, thus.
  SECOND LORD. They every villain of the fight.
  Ham. That's the meaun was the King by the Duke of Gloucester's fall he
    say                                          "
  LORENZO
  OTHELLO. There is on some faint means in his best,
    That she dies to-stol'n thy youngly by the gods,
    The leaves that saying bury and your Romeo.
  KING 

In [14]:
print(sample(rnn, char_encoder, 1000, prime_str='HAMLET.', temperature=.8))

HAMLET. A ront was even to so to me.
  GLOUCESTER. What mamone, almost so tell my shore, my daughter?
     In my gold to her with ears; the gods,
    Whose ground, and so how he shall say sweet monk.
  MENENIUS. Was not you will can decreee,
    But that has too grown. So wear yourself, to his wit,
    This second's hand; you are the letter.
    They live the treason for mother, or he speaks
    With lost of first soon of burd'ny faith,
    To make me brave their counsel.
    Would One manner before his wit
    Than you have honour and make a question;
    A midness. What stack you her brother, and my love to me.
    A song; then by your willing to some el them have been but signify
    He was had such enemy.
    But I live into a tumn'd a fives to the manar hour. Why, find the country of Page,
    Who shall be bare what    And thou wilt have the do fight
    And for thine own cankerling world.
    Is he been of the blessing of duty,
    And so the remembrance of breath.
    The Capili

In [0]:
print(sample(rnn, encoder=char_encoder, size=1000, prime="IAGO."))

IAGO. The gentlemen lie up my like the case.
    And she crust to kill him to kish  the anger
    her company, course, to him; but this nest;
    And when I met you here haste with him.
  MALVOLIO. Here we muke the liver beholds all; but the shadows; and my brother Henryes.
  THIRD SERVANT. And what then there?
    Will you not know your speech in Eesuafiches?
  Ham. How is it her hear so say                                                   Enter Member's tongues on
    Under or with when they does alone a life,
    But no bold another, the Edward go to
                                                                                      Exeunt BASTARD. At, send the reason and sleeps meats of her, and the word besin but I say
    Than the crunds in man's sister.
  CLAUDIO. Our expeding, I will sent us of your satisful sin,
    Is the into his black of Clarence' down
    me, and wearing all worse is heavile with her.
  CARDINAL. What all the world, so stay?
  DEMETRITA. Speak is away i

In [0]:
#temperature = 0.8
print(sample(rnn, encoder=char_encoder, size=600, prime="IAGO."))

IAGO. Even the sins. She breaks her homes down me, for their pains
    Of a money die a beg- as I had washes deliver'd of labour
    shame to the times in these treasurghs with them of my business.  
                                                     Enter DUKE OF Engeen in the death.]
  PROSPERO. If I see the state!- Do you like some decked samprise,
    When it was by the prince blenty like and buttest
    Make no cause thou neither,
   Against them to you, and by his countrymen.
                                                                                                                     
